## 1. Kensu initialisation

To use the library, you need to add the kensu jar to the spark client

Initialize the client with the `KensuProvider` object with its `Context`:
- `process_name`:  the application name
- `project_names`: where the application is running 
- `environment`: ... well

In [1]:
%%init_spark
launcher.jars = ["libs/kensu-dam-spark-collector-0.17.2_spark-3.0.1.jar","libs/sdk_2.12.jar"]
launcher.conf.set("spark.sql.shuffle.partitions", "1")

In [2]:
val app = 1
val week = 1

Intitializing Scala interpreter ...

Spark Web UI available at http://jupyter-kensuio-2doss-2ddodd-2dworkshop-2dr10bzxei:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1636438800758)
SparkSession available as 'spark'


app: Int = 1
week: Int = 1


In [3]:
implicit val ch = new io.kensu.dodd.sdk.ConnectHelper(s"./conf/application${app}-week${week}.properties")
val dataFolder = "data"+ch.properties("dam.activity.user").toString
io.kensu.third.integration.TimeUtils.setMockedTime(ch.properties("mocked.timestamp").asInstanceOf[Long], false)
io.kensu.dam.lineage.spark.lineage.Implicits.SparkSessionDAMWrapper(spark).track(ch.properties.get("dam.ingestion.url").map(_.toString), None)(ch.properties.toList:_*)

maybeDamFileDebugLevel: Some(INFO)
Activating DAM debug logging to file: debug.log
Done activating DAM debug logging to a file
Creating a new DAM scala client actor-system
Done creating a new DAM scala client actor-system.


ch: io.kensu.dodd.sdk.ConnectHelper = io.kensu.dodd.sdk.ConnectHelper@53517333
dataFolder: String = datamyname
res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@b9c5fc9


 ## 2. Execution of the pipeline - App 1 : Join
 
 This application is joining datasource `customer` and `transaction`


In [5]:
val df_customer = spark.read.option("inferSchema","true").option("header","true").load(s"./data_${ch.properties("dam.activity.user")}/week${week}/custinfo")

df_customer: org.apache.spark.sql.DataFrame = [id: int, first_name: string ... 3 more fields]


In [6]:
val df_transaction = spark.read.option("inferSchema","true").option("header","true").load(s"./data_${ch.properties("dam.activity.user")}/week${week}/transactions")

df_transaction: org.apache.spark.sql.DataFrame = [id: int, date: double ... 3 more fields]


In [7]:
val df_join = df_customer.join(df_transaction, df_customer("id") === df_transaction("id"),"inner").drop(df_transaction("id"))

df_join: org.apache.spark.sql.DataFrame = [id: int, first_name: string ... 7 more fields]


In [8]:
df_join.count()

res1: Long = 1000


In [9]:
val sdk = new io.kensu.dodd.sdk.SDK(ch.properties("kensu.sdk.url").toString, ch.properties("kensu.pat").toString)

sdk: io.kensu.dodd.sdk.SDK = io.kensu.dodd.sdk.SDK@4884e416


In [ ]:
sdk.range()

In [ ]:
df_join.write.mode("overwrite").save(s"./data_${properties("dam.activity.user")}/week${week}/joined_data")